In [5]:
# - *- coding: utf- 8*-
from LambdaNodeEmbedding import GetKerasNAP, KerasEval
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.callbacks import History

#These are the edges between the nodes.
edgelist = np.array([
                    [0,1,1,1],
                    [0,0,0,1],
                    [0,1,0,0],
                    [0,0,0,0]
                    ], dtype='float32')

edgelist_ten = K.variable(value=edgelist, dtype='float32')
new_x_x_x = tf.concat([edgelist_ten[0,:],edgelist_ten[1,:]], 0)

def GetXDepth(matrix):
    if len(matrix.shape) > 0:
        return matrix.shape[0]
    else:
        return 1

def GetYDepth(matrix):
    if len(matrix.shape) > 1:
        return matrix.shape[1]
    else:
        return 1

def BuildZeroedConcatRow(edge_row, index):
    zero_calc = tf.zeros((index,), tf.float32)    
    keep_calc =  tf.ones(((edge_row.shape[0]-index),), tf.float32)
    concat_calc = tf.concat([zero_calc,keep_calc],0)
    result = tf.multiply(concat_calc, edge_row) 
    return result

def IndexOutOfBounds(edge_row, index):
    if index <= edge_row.shape[0]:
        return BuildZeroedConcatRow(edge_row, index)
    else:
        return tf.zeros(edge_row.shape, tf.float32) 

def ZerofyUnderIndexInRow(edge_row, index):
    if index < 1:
        return edge_row
    else: 
        return IndexOutOfBounds(edge_row, index)

def ZerofyUnderPrincipalDiagonal(edge_emb, result_emb, iterator):
    
    #Condition => if current iteration is lower as max iterations
    def cond(edge_emb, result_emb, i):
         return tf.less(i, iterator)

    #Loop Body Processing Unit => get all connection dependencies
    def body(edge_emb, result_emb, i):
        
        print(i)
        result_row = ZerofyUnderIndexInRow(edge_emb[i,:], i)
        
        #Store result in Tensorarray result_emb
        result_emb = result_row.write(i, [result_row])
        
        #Add +1 to Iterator and return process result!
        return [edge_emb, result_emb, tf.add(i, 1)]

    #return the loop setup
    return tf.while_loop(cond, body, [edge_emb, result_emb, 0])


init = tf.global_variables_initializer()


In [6]:
with tf.Session() as sess:
    init.run()

    print('New_x_d: ',GetXDepth(new_x_x_x))
    print('New_y_d: ',GetYDepth(new_x_x_x))
    print('El_x_d: ',GetXDepth(edgelist_ten))
    print('El_y_d: ',GetYDepth(edgelist_ten))

    print(KerasEval(BuildZeroedConcatRow(new_x_x_x, 1)))
    print('\n#######\n')
    print(KerasEval(BuildZeroedConcatRow(new_x_x_x, 3)))
    print('\n#######\n')
    print(KerasEval(BuildZeroedConcatRow(new_x_x_x, 5)))
    print('\n#######\n')
    print(KerasEval(BuildZeroedConcatRow(new_x_x_x, 8)))

New_x_d:  8
New_y_d:  1
El_x_d:  4
El_y_d:  4
[0. 1. 1. 1. 0. 0. 0. 1.]

#######

[0. 0. 0. 1. 0. 0. 0. 1.]

#######

[0. 0. 0. 0. 0. 0. 0. 1.]

#######

[0. 0. 0. 0. 0. 0. 0. 0.]


In [7]:
with tf.Session() as sess:
    init.run()
    
    print('\n#######')
    print('#######')
    print('#######\n')
    
    print(KerasEval(ZerofyUnderIndexInRow(new_x_x_x, -1)))
    print('\n#######\n')
    print(KerasEval(ZerofyUnderIndexInRow(new_x_x_x, 1)))
    print('\n#######\n')
    print(KerasEval(ZerofyUnderIndexInRow(new_x_x_x, 3)))
    print('\n#######\n')
    print(KerasEval(ZerofyUnderIndexInRow(new_x_x_x, 9)))


#######
#######
#######

[0. 1. 1. 1. 0. 0. 0. 1.]

#######

[0. 1. 1. 1. 0. 0. 0. 1.]

#######

[0. 0. 0. 1. 0. 0. 0. 1.]

#######

[0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
with tf.Session() as sess:
    init.run()
    
    print('\n#######')
    print('#######')
    print('#######\n')
    
    print(KerasEval(ZerofyUnderPrincipalDiagonal(edgelist_ten,edgelist_ten,edgelist_ten.shape[1])))


#######
#######
#######

Tensor("while_1/Identity_2:0", shape=(), dtype=int32)


TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.